# Figure Notebook

In [27]:
"""
Import needed libraries
"""

import numpy as np
import pandas as pd
from copy import copy
import sys
import itertools
from netCDF4 import Dataset

"""
Import all from projects python scripts
"""

from geomip_data_smb import *
from analysis import *
from plotting import *

In [6]:
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 300 # set inline images to hi-res
%matplotlib inline

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
def get_2d_geomip_new(var, model, exp, run, seas, stat, time='11-50'):
    
    """
    Returns array of standard GeoMIP netcdf file from my archive using Dataset.
    Screens out common dimension names to return a single variable as an array.
    Will fail if there are more than one variables or a dimension has been missed.
    """
    
    # Define nc_file format
    nc_file_base = "{var}_{model}_{exp}_{run}_{time}_{seas}_{stat}.nc"
    nc_file = nc_file_base.format(var=var, model=model, exp=exp, run=run, time=time, seas=seas, stat=stat)

    # Define directory format
    in_dir_base = "/n/home03/pjirvine/keithfs1_pji/geomip_archive/final_data/{model}/{exp}/time{stat}/"
    in_dir = in_dir_base.format(model=model, exp=exp, stat=stat)
    
    file_loc = in_dir + nc_file

    # function which removes list from list.
    def take_list_from_list(longlist, list2remove):
        for item in list2remove:
            try:
                longlist.remove(item)
            except:
                pass # or say something...
        
    dim_list = ['lon', 'lon_bnds', 'lat', 'lat_bnds', 'time', 'time_bnds', u'longitude', u'latitude', u'ht', u't', u't_bnds']
    
    if os.path.isfile(file_loc):
        nc = Dataset(file_loc)
        
        vars_dims_in_nc = nc.variables.keys() # list vars and dims
        vars_in_nc = copy(vars_dims_in_nc)
        
        # remove dims from vars_dims to leave vars only
        take_list_from_list(vars_in_nc,dim_list)
        
        if len(vars_in_nc) == 1:
            var_out = vars_in_nc[0]
            return nc.variables[var_out][:]
        else:
            print "more than one var",nc_file
            return None
    
    else:
        return None # doesn't make print statement as many files are missing.
    

In [ ]:
"""
Set up variable and other lists
"""

model_list = ['GISS-E2-R','HadCM3', 'BNU-ESM', 'CCSM4', 'CESM-CAM5.1-FV', 'CanESM2', 'CSIRO-Mk3L-1-2','HadGEM2-ES' ,'IPSL-CM5A-LR','MIROC-ESM','MPI-ESM-LR','NorESM1-M']
seas_list = ['ann','jja','djf']
exp_list = ['piControl','abrupt4xCO2','G1']

var_list, var_offsets, var_mults, var_name_mod = [], {}, {}, {}

s2d = 60.0*60.0*24.0
k2c = -273.15

def add_var(var_name, offset, mult, name_mod):
    var_list.append(var_name)
    var_offsets[var_name] = offset
    var_mults[var_name] = mult
    var_name_mod[var_name] = name_mod

# Climate stuff
add_var('tas_Amon',k2c,1.0, "")         # 2m temp
add_var('pr_Amon', 0.0, s2d, "")            # Precip - evap
add_var('p-e_Amon', 0.0, s2d, "")            # Precip - evap
add_var('prsn_Amon', 0.0, s2d, "")          # snowfall
add_var('evspsbl_Amon', 0.0, s2d, "")       # total evap
add_var('sbl_Amon', 0.0, s2d, "")           # sublimation
add_var('ts_Amon', k2c, 1.0, "")        # surface temp
add_var('hurs_Amon', 0.0, 1.0, "")          # relative humidity
# Energy flux at surface
add_var('rlus_Amon',0.0,1.0,"")             # LW up 
add_var('rlds_Amon',0.0,1.0,"")             # LW down
add_var('rsus_Amon',0.0,1.0,"")             # SW up
add_var('rsds_Amon',0.0,1.0,"")             # SW down
add_var('hfss_Amon',0.0,1.0,"")             # sensible heat up
add_var('hfls_Amon',0.0,1.0,"")             # latent heat up
# Snow Stuff
add_var('lwsnl_LImon',0.0,1.0,"")           # liquid water snow
add_var('snd_LImon',0.0,1.0,"")             # fresh snow thickness
add_var('snm_LImon',0.0,s2d,"")             # snow melt
add_var('hfdsn_LImon',0.0,1.0,"")           # heat flux into snow
add_var('tsn_LImon',k2c,1.0,"")         # snow temperature
add_var('snw_LImon',0.0,1.0,"")             # snow mass


In [ ]:
"""
Load up masks
"""

masks_dict = {}
weights_dict = {}

mask_list = ['global','land_noice','greenland','antarctica']

for model in model_list:
    
    model_masks = get_masks_weights(model)
    
    for mask in mask_list:
        
        masks_dict[model, mask] = model_masks[1][mask]
        weights_dict[model, mask] = model_masks[2][mask+'_area']

In [ ]:
"""
Load up variables
"""

means_dict = {}
stds_dict = {}

# 4 nested for-loops squashed into one!
for var, model, exp, seas in itertools.product(var_list, model_list, exp_list, seas_list):
               
    # look up run for model-exp combo
    run = model_exp_runs[model+'_'+exp][0]

    try:
        means_dict[var, model, exp, seas] = var_mults[var] * get_2d_geomip(var, model, exp, run, seas, 'mean') + var_offsets[var]
    except:
        means_dict[var, model, exp, seas] = None
    try:
        stds_dict[var, model, exp, seas] = var_mults[var] * get_2d_geomip(var, model, exp, run, seas, 'std')
    except:
        stds_dict[var, model, exp, seas] = None

In [ ]:
"""
create surface heat variables
"""

# add surface heat to means_dict
for model, exp, seas in itertools.product(model_list, exp_list, seas_list):
    
    # load all vars
    rsds = means_dict['rsds_Amon', model, exp, seas]
    rsus = means_dict['rsus_Amon', model, exp, seas]
    rlds = means_dict['rlds_Amon', model, exp, seas]
    rlus = means_dict['rlus_Amon', model, exp, seas]
    hfss = means_dict['hfss_Amon', model, exp, seas]
    hfls = means_dict['hfls_Amon', model, exp, seas]
    
    sfc_vars = [rsds,rsus,rlds,rlus,hfss,hfls]
    
    # calculate
    if any(type(t) == type(None) for t in sfc_vars):
        sfc_heat, sfc_rad, sfc_nonrad = None, None, None
    else:
        sfc_heat = rsds + rlds - rsus - rlus - hfss - hfls
        sfc_rad = rsds + rlds - rsus - rlus
        sfc_nonrad = - hfss - hfls
        sfc_sw = rsds - rsus
        sfc_lw = rlds - rlus
    
    # add vars
    means_dict['sfc_heat_Amon', model, exp, seas] = sfc_heat
    means_dict['sfc_rad_Amon', model, exp, seas] = sfc_rad
    means_dict['sfc_nonrad_Amon', model, exp, seas] = sfc_nonrad
    means_dict['sfc_sw_Amon', model, exp, seas] = sfc_sw
    means_dict['sfc_lw_Amon', model, exp, seas] = sfc_lw
    
var_list.extend(['sfc_heat_Amon','sfc_rad_Amon','sfc_nonrad_Amon','sfc_sw_Amon','sfc_lw_Amon'])

## Rescaled results to restore global, greenland and antarctic temperatures to control

# Analysis Begins

In [ ]:
# apply mask to data
def masked_sum(var,model,exp,seas,mask):
    
    # if exception is thrown return NAN
    try:
        data = np.sum(means_dict[var, model,exp,seas] * weights_dict[model,mask])
    except:
        data = np.nan
        
    return data

# Big Table
The goal is to produce a big table of results that show picontrol and both anomalies.
Each table will have rows of model results and columns of var_anom pairs. Tables will be produced for each mask-season combo.

In [ ]:
def add_mean_median(dict_in):
    dict_in['_mean'] = np.nanmean(dict_in.values())
    dict_in['_median'] = np.nanmedian(dict_in.values())

In [ ]:
table_dir = "/n/home03/pjirvine/projects/GeoMIP_SMB/tables/"

# dictionary to store dataframes / tables
df_dict = {}

# One table for each mask-seas combo
for mask, seas in itertools.product(mask_list, seas_list):

    var_anom_dict = {}

    for var in var_list:

        pic_dict, CO2_anom_dict, G1_anom_dict = {}, {}, {}

        #Calculate and store control and anom values per model
        for model in model_list:

            pic_dict[model] = masked_sum(var,model,'piControl',seas,mask)
            CO2_anom_dict[model] = masked_sum(var,model,'abrupt4xCO2',seas,mask) - masked_sum(var,model,'piControl',seas,mask)
            G1_anom_dict[model] = masked_sum(var,model,'G1',seas,mask) - masked_sum(var,model,'piControl',seas,mask)

        add_mean_median(pic_dict)
        add_mean_median(CO2_anom_dict)
        add_mean_median(G1_anom_dict)
        
        var_short = var.replace('_Amon', '').replace('_LImon', '')
        var_anom_dict[var_short+'_pic'] = pic_dict
        var_anom_dict[var_short+'_CO2_anom'] = CO2_anom_dict
        var_anom_dict[var_short+'_G1_anom'] = G1_anom_dict

    # Convert dictionary to dataframe, store and also output as csv table.
    df = pd.DataFrame.from_dict(var_anom_dict)
    df_dict[mask,seas] = df
    df.to_csv(table_dir+'{mask}_{seas}.csv'.format(mask=mask,seas=seas))

In [ ]:
# # Extract dataframe like this:
# print df_dict['global','ann']
# # Extract columns from dataframe like this:
# print df_dict['global','ann']['evspsbl_CO2_anom']
# # Extract list of values like this:
# print df_dict['global','ann']['evspsbl_CO2_anom'].values

# Mean results

In [ ]:
# get dataframes
globe = df_dict['global','ann']
land_noice = df_dict['land_noice','ann']
greenland = df_dict['greenland','ann']
antarctica = df_dict['antarctica','ann']
gr_summer = df_dict['greenland','jja']
ant_summer = df_dict['antarctica','djf']

## global energetics
Global surface heat budget is not balanced in piControl, all models are taking up heat into the ocean and land. GISS has a very large energy imbalance!

In [ ]:
globe[['sfc_heat_pic','sfc_heat_CO2_anom', 'sfc_heat_G1_anom']]

In [ ]:
land_noice[['sfc_heat_pic','sfc_heat_CO2_anom', 'sfc_heat_G1_anom']]

In [ ]:
globe[['tas_pic','tas_CO2_anom', 'tas_G1_anom']]

### modest evidence of over-effectiveness on total heat flux
The energy budget over Greenland and Antartcica is different between models, some show modest heat fluxes in and others show modest heat fluxes out. Median control heat fluxes are GR: -0.4 Wm-2 and ANT: -0.7, for CO2 there is a considerable warming of Greenland GR: 3.9 Wm-2, and little effect on antarctica ANT: 0.4 Wm-2, for G1 there is a modest reduction compared to control GR: -0.03, ANT: -0.05

In [ ]:
greenland[['sfc_heat_pic','sfc_heat_CO2_anom', 'sfc_heat_G1_anom']]

In [ ]:
antarctica[['sfc_heat_pic','sfc_heat_CO2_anom', 'sfc_heat_G1_anom']]

### on radiative side, there is a considerable heating compared to control
Both greenland and antarctica are radiatively cooling to the tune of 15-20 WM-2 in control (heated by specific and latent heating to "balance"), at 2xCO2 Greenland heats by ~6 Wm-2 and Ant by ~2.5 Wm-2. this is reduced to ~0.7Wm-2 and ~0.5 Wm-2 in G1.

In [ ]:
greenland[['sfc_rad_pic','sfc_rad_CO2_anom', 'sfc_rad_G1_anom']]

In [ ]:
antarctica[['sfc_rad_pic','sfc_rad_CO2_anom', 'sfc_rad_G1_anom']]

### heat fluxes
in picontrol heat fluxes warm Greenland and Antarctica to tune of ~15 Wm-2. There is a reduction in heat fluxes into greenland and antarctica in 2xCO2, GR:-1.5, ANT:-2.2, partially offsetting increased radiative heating in Greenland and mostly offsetting it in Antarctica. in G1 the reduction in heat fluxes is moderated, GR:-0.8, ANT:-0.7.

In [ ]:
greenland[['sfc_nonrad_pic','sfc_nonrad_CO2_anom', 'sfc_nonrad_G1_anom']]

In [ ]:
antarctica[['sfc_nonrad_pic','sfc_nonrad_CO2_anom', 'sfc_nonrad_G1_anom']]

## Accumulalation
For most models ~90% of precip is as snow on greenland and ~100% for Antarctica in control. P-E is close to P in all models, due to low total evap rates.

In [ ]:
greenland[['pr_pic','prsn_pic','p-e_pic']]

In [ ]:
antarctica[['pr_pic','prsn_pic','p-e_pic']]

### precip change
GR and ANT both see a significant increase in precip at high CO2. This is basically completely offset in G1.

In [ ]:
greenland[['pr_pic','pr_CO2_anom','pr_G1_anom']]

In [ ]:
antarctica[['pr_pic','pr_CO2_anom','pr_G1_anom']]

### lying snow

In [ ]:
greenland[['snw_pic','snw_CO2_anom','snw_G1_anom']]

In [ ]:
antarctica[['snw_pic','snw_CO2_anom','snw_G1_anom']]

## Summer changes

### SAT
Most of summer temp increase is offset by G1, though somewhat more effectively in Ant.

In [ ]:
gr_summer[['tas_pic','tas_CO2_anom','tas_G1_anom']]

In [ ]:
ant_summer[['tas_pic','tas_CO2_anom','tas_G1_anom']]

### snow melt
Increases in snow melt are generally offset but large differences between models and few models with data makes it tricky to be certain.

In [ ]:
gr_summer[['snm_pic','snm_CO2_anom','snm_G1_anom']]

In [ ]:
ant_summer[['snm_pic','snm_CO2_anom','snm_G1_anom']]

### radiative changes in Greenland summer
The large increase in summer sfc heat flux into the ice is generally offset in G1 +0.08Wm-2, there is a somewhat greater reduction in summer surface radiation which ends somewhat below control -0.4 Wm-2, offset by greater LH and SH heat fluxes. There is a significant reduction in net SW at surface -2.5 Wm-2 that is offset by greater LW at surface +2.0 Wm-2

In [ ]:
gr_summer[['sfc_heat_pic','sfc_heat_CO2_anom','sfc_heat_G1_anom']]

In [ ]:
gr_summer[['sfc_rad_pic','sfc_rad_CO2_anom','sfc_rad_G1_anom']]

In [ ]:
gr_summer[['sfc_sw_pic','sfc_sw_CO2_anom','sfc_sw_G1_anom']]

In [ ]:
gr_summer[['sfc_lw_pic','sfc_lw_CO2_anom','sfc_lw_G1_anom']]

### radiative changes in Antarctic summer
slight net reduction in ANT sfc heat -0.2 Wm-2, explained by rad -0.15 Wm-2, -3.0 Wm-2 sfc SW mostly offset by LW increase +2.85 Wm-2

In [ ]:
ant_summer[['sfc_heat_pic','sfc_heat_CO2_anom','sfc_heat_G1_anom']]

In [ ]:
ant_summer[['sfc_rad_pic','sfc_rad_CO2_anom','sfc_rad_G1_anom']]

In [ ]:
ant_summer[['sfc_sw_pic','sfc_sw_CO2_anom','sfc_sw_G1_anom']]

In [ ]:
ant_summer[['sfc_lw_pic','sfc_lw_CO2_anom','sfc_lw_G1_anom']]

In [ ]:
print globe.columns.values

### Variables for reference

In [ ]:
# Climate stuff
add_var('tas_Amon',k2c,0.0, "")         # 2m temp
add_var('pe_Amon', 0.0, s2d, "")            # Precip
add_var('prsn_Amon', 0.0, s2d, "")          # snowfall
add_var('evspsbl_Amon', 0.0, s2d, "")       # total evap
add_var('sbl_Amon', 0.0, s2d, "")           # sublimation
add_var('ts_Amon', k2c, 0.0, "")        # surface temp
add_var('hurs_Amon', 0.0, 0.0, "")          # relative humidity
# Energy flux at surface
add_var('rlus_Amon',0.0,0.0,"")             # LW up 
add_var('rlds_Amon',0.0,0.0,"")             # LW down
add_var('rsus_Amon',0.0,0.0,"")             # SW up
add_var('rsds_Amon',0.0,0.0,"")             # SW down
add_var('hfss_Amon',0.0,0.0,"")             # sensible heat up
add_var('hfls_Amon',0.0,0.0,"")             # latent heat up
means_dict['sfc_heat_Amon', model, exp, seas] = sfc_heat
means_dict['sfc_rad_Amon', model, exp, seas] = sfc_rad
# Snow Stuff
add_var('lwsnl_LImon',0.0,0.0,"")           # liquid water snow
add_var('snd_LImon',0.0,0.0,"")             # fresh snow thickness
add_var('snm_LImon',0.0,s2d,"")             # snow melt
add_var('hfdsn_LImon',0.0,0.0,"")           # heat flux into snow
add_var('tsn_LImon',k2c,0.0,"")         # snow temperature
add_var('snw_LImon',0.0,0.0,"")             # snow mass